In [7]:
 #Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [8]:
train_df = pd.read_csv('/workspaces/codespaces-blank/news_classification/data/train.csv')
train_df

,ArticleId,Text,Category
0,1155,chancellor rallies labour voters gordon brown ...,politics
1,1980,india s maruti sees profits jump india s bigge...,business
2,386,ukip s secret weapon by any measure new york...,politics
3,1436,banker loses sexism claim a former executive a...,business
4,304,dallaglio eyeing lions tour place former engla...,sport
...,...,...,...
1038,464,dozens held over id fraud site twenty-eight pe...,tech
1039,1468,commodore finds new lease of life the once-fam...,tech
1040,1023,fockers keeps us box office lead film comedy m...,entertainment
1041,1771,astrazeneca hit by drug failure shares in angl...,business


In [9]:
train_df.describe()

,ArticleId
count,1043.000000
mean,1118.684564
std,644.332014
min,2.000000
25%,569.500000
50%,1106.000000
75%,1684.000000
max,2224.000000


In [14]:
train_df.dtypes

ArticleId     int64
Text         object
Category     object
dtype: object

In [11]:
train_df.isnull().sum()

ArticleId    0
Text         0
Category     0
dtype: int64